# 1. Importação das bibliotecas, definição dos métodos e tratamento inicial




In [222]:
%%capture
!pip install ray
!pip install unicode
!pip install unidecode

In [223]:
import pandas as pd
import ray
import plotly.express as px
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import numpy as np
import unidecode
from scipy import stats
from google.colab import drive
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [224]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [225]:
def find_best_lambda(data, lambda_range):
    best_lambda = None
    best_llf = -np.inf  # initialize with negative infinity

    for lmbda in lambda_range:
        llf = stats.yeojohnson_llf(lmbda, data)
        if llf > best_llf:
            best_llf = llf
            best_lambda = lmbda

    return best_lambda

In [226]:
def remove_accents(a):
  a = a.encode('utf-8', errors='ignore')
  return unidecode.unidecode(a.decode('utf-8'))

In [227]:
cadunico = pd.read_csv("/content/drive/MyDrive/Hunger prediction/data/cadunico_data.csv")
sisvan = pd.read_csv("/content/drive/MyDrive/Hunger prediction/data/sisvan.csv")

In [228]:
sisvan.replace(['%', '-'], ['', 0], regex=True, inplace=True)
sisvan['Município'] = sisvan['Município'].apply(remove_accents)

for col in sisvan.columns[1:]:
  sisvan[col] = sisvan[col].astype(float)

sisvan['Município'] = sisvan['Município'].str.lower()

In [229]:
cadunico['Município'] = cadunico['Município'].str.lower()
cadunico['Município'] = cadunico['Município'].apply(remove_accents)

In [230]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
cadunico

,Município,Faixa da renda total da família / Até 1 S.M.,Faixa da renda total da família / Entre 1 e 2 S.M.,Faixa da renda total da família / Entre 2 e 3 S.M.,Faixa da renda total da família / Acima de 3 S.M.,Faixa da renda familiar per capita / Pobreza 1 (até R$ 109),Faixa da renda familiar per capita / Pobreza 2 (de R$ 109 a R$ 218),Faixa da renda familiar per capita / Baixa Renda,Faixa da renda familiar per capita / Acima de 1/2 S.M.,Recebe PBF família / Não,Recebe PBF família / Sim,Calçamento em frente ao seu domicílio / Total,Calçamento em frente ao seu domicílio / Parcial,Calçamento em frente ao seu domicílio / Não existe,Situação do domicílio / Urbanas,Situação do domicílio / Rurais,Espécie do domicílio / Particular Permanente,Espécie do domicílio / Particular improvisado,Espécie do domicílio / Coletivo,Existência de banheiro / Sim,Existência de banheiro / Não,Forma de abastecimento de água / Rede geral de distribuição,Forma de abastecimento de água / Poço ou nascente,Forma de abastecimento de água / Cisterna,Forma de abastecimento de água / Outra forma,Forma de coleta do lixo / É coletado diretamente,Forma de coleta do lixo / É coletado indiretamente,Forma de coleta do lixo / É queimado ou enterrado na propriedade,"Forma de coleta do lixo / É jogado em terreno baldio ou logradouro (rua, avenida, etc.)",Forma de coleta do lixo / É jogado em rio ou mar,Forma de coleta do lixo / Tem outro destino,Forma de escoamento sanitário / Rede coletora de esgoto ou pluvial,Forma de escoamento sanitário / Fossa séptica,Forma de escoamento sanitário / Fossa rudimentar,Forma de escoamento sanitário / Vala a céu aberto,"Forma de escoamento sanitário / Direto para um rio, lago ou mar",Forma de escoamento sanitário / Outra forma,Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento,Material predominante nas paredes externas do domicílio / Alvenaria/tijolo sem revestimento,Material predominante nas paredes externas do domicílio / Madeira aparelhada,Material predominante nas paredes externas do domicílio / Taipa revestida,Material predominante nas paredes externas do domicílio / Taipa não revestida,Material predominante nas paredes externas do domicílio / Madeira aproveitada,Material predominante nas paredes externas do domicílio / Palha,Material predominante nas paredes externas do domicílio / Outro Material,Material predominante no piso do domicílio / Terra,Material predominante no piso do domicílio / Cimento,Material predominante no piso do domicílio / Madeira aproveitada,Material predominante no piso do domicílio / Madeira aparelhada,"Material predominante no piso do domicílio / Cerâmica, lajota ou pedra",Material predominante no piso do domicílio / Carpete,Material predominante no piso do domicílio / Outro Material,Água canalizada no domicílio / Sim,Água canalizada no domicílio / Não,Tipo de iluminação / Elétrica com medidor próprio,Tipo de iluminação / Elétrica com medidor comunitário,Tipo de iluminação / Elétrica sem medidor,"Tipo de iluminação / Óleo, querosene ou gás",Tipo de iluminação / Vela,Tipo de iluminação / Outra forma,Recebe PBF família / Não.1,Recebe PBF família / Sim.1,Pessoa com marcação de trabalho infantil / Sim,Pessoa com marcação de trabalho infantil / Não,Não recebe ajuda de terceiros / Opção não marcada no formulário,Não recebe ajuda de terceiros / Opção marcada no formulário,Curso mais elevado que a pessoa frequentou / Creche,Curso mais elevado que a pessoa frequentou / Pré-escola (exceto CA),Curso mais elevado que a pessoa frequentou / Classe de Alfabetização - CA,"Curso mais elevado que a pessoa frequentou / Ensino Fundamental 1ª a 4ª séries, Elementar (Primário), Primeira fase do 1º grau","Curso mais elevado que a pessoa frequentou / Ensino Fundamental 5ª a 8ª séries, Médio 1º ciclo (Ginasial), Segunda fase do 1º grau",Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos),Curso mais elevado que a pessoa frequentou / Ensino Fundamental Especial,"Cur

# 2. Preprocessamento

## 2.1. PCA

In [231]:
cadunico_copia = cadunico.iloc[:, 1:].copy()

# Normalizar os dados usando StandardScaler
scaler = StandardScaler()
normalized_data = scaler.fit_transform(cadunico_copia)

# Aplicar o PCA
pca = PCA()
principal_components = pca.fit_transform(normalized_data)

In [232]:
len(principal_components)

184

In [233]:
# O objeto "principal_components" agora contém os componentes principais
# Você pode acessar os componentes e a variância explicada da seguinte forma:
explained_variance = pca.explained_variance_ratio_
components = pca.components_
cumulative_variance = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

print("Soma total da variância explicada: ", sum(explained_variance), '\n')
print("Proporção da variância 1o primeiros PCs: \n", list(explained_variance[0:10]), '\n')
print("Varância acumulada:\n", cumulative_variance)

Soma total da variância explicada:  1.0 

Proporção da variância 1o primeiros PCs: 
 [0.14091691009474028, 0.08777553508422328, 0.05056996301169666, 0.04437054165813983, 0.0406189857464161, 0.03999541348786884, 0.03188308040228925, 0.02827804748605796, 0.027552536515984705, 0.024733043534982457] 

Varância acumulada:
 [ 14.09  22.87  27.93  32.37  36.43  40.43  43.62  46.45  49.21  51.68
  53.99  56.27  58.43  60.48  62.28  64.    65.68  67.2   68.67  70.11
  71.48  72.8   74.05  75.19  76.29  77.38  78.44  79.48  80.46  81.41
  82.34  83.24  84.07  84.87  85.63  86.35  87.07  87.74  88.4   88.99
  89.56  90.09  90.61  91.11  91.61  92.09  92.54  92.97  93.39  93.79
  94.15  94.51  94.86  95.19  95.51  95.81  96.09  96.36  96.62  96.87
  97.1   97.32  97.54  97.74  97.93  98.12  98.29  98.45  98.6   98.74
  98.87  98.99  99.11  99.22  99.32  99.41  99.49  99.56  99.63  99.69
  99.74  99.79  99.83  99.87  99.91  99.94  99.96  99.98 100.   100.01
 100.02 100.02 100.02 100.02 100.02 100.0

In [234]:
x = list(range(1, len(cumulative_variance)+1))
fig = go.Figure(data=go.Scatter(x = x, y = cumulative_variance, mode='lines+markers'))
fig.update_layout(
    xaxis=dict(title='PCs', dtick=10 ,showgrid=True),
    yaxis=dict(title='Cumulative variance',  dtick=10, showgrid=True)
)
fig.show()

In [235]:
def PCA_scatter(pcs, loadings, target, column_name, target_name='value', labels=None):
    xs = pcs[:,0]
    ys = pcs[:,1]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())

    pc1 = xs*scalex
    pc2 = ys*scaley

    df = {'pc1': pc1, 'pc2': pc2, target_name: target}
    df = pd.DataFrame(data=df)

    fig = px.scatter(df, x='pc1', y='pc2', color=target_name, color_continuous_scale = "Bluyl")

    # for i, feature in enumerate(column_name):
    #     fig.add_shape(type='line',
    #                   x0=0,
    #                   y0=0,
    #                   x1=loadings[i,0],
    #                   y1=loadings[i,1]
    #                 )
    #     fig.add_annotation(x=loadings[i,0],
    #                        y=loadings[i,1],
    #                        ax=0,
    #                        ay=0,
    #                        xanchor="center",
    #                        yanchor="bottom",
    #                        text = feature,
    #                        font=dict(size=18,
    #                                  color="crimson"
    #                                 )
    #                       )
    fig.show()

#Call the function. Use only the 2 PCs.
PCA_scatter(principal_components[:,0:2], loadings = np.transpose(pca.components_[0:2, :]),
            target = sisvan[sisvan.columns[3]], column_name = cadunico.columns[1:],
            target_name = sisvan.columns[3])

In [236]:
cadunico_pca = pd.DataFrame()
cadunico_pca["Município"] = cadunico["Município"]

for i, val in enumerate(np.transpose(principal_components)[0:]):
  cadunico_pca['pc_' + str(i+1)] = val

<ipython-input-236-5f8433d19f17>:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-236-5f8433d19f17>:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-236-5f8433d19f17>:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-236-5f8433d19f17>:5: PerformanceWarning:

In [237]:
cadunico_pca

,Município,pc_1,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7,pc_8,pc_9,pc_10,pc_11,pc_12,pc_13,pc_14,pc_15,pc_16,pc_17,pc_18,pc_19,pc_20,pc_21,pc_22,pc_23,pc_24,pc_25,pc_26,pc_27,pc_28,pc_29,pc_30,pc_31,pc_32,pc_33,pc_34,pc_35,pc_36,pc_37,pc_38,pc_39,pc_40,pc_41,pc_42,pc_43,pc_44,pc_45,pc_46,pc_47,pc_48,pc_49,pc_50,pc_51,pc_52,pc_53,pc_54,pc_55,pc_56,pc_57,pc_58,pc_59,pc_60,pc_61,pc_62,pc_63,pc_64,pc_65,pc_66,pc_67,pc_68,pc_69,pc_70,pc_71,pc_72,pc_73,pc_74,pc_75,pc_76,pc_77,pc_78,pc_79,pc_80,pc_81,pc_82,pc_83,pc_84,pc_85,pc_86,pc_87,pc_88,pc_89,pc_90,pc_91,pc_92,pc_93,pc_94,pc_95,pc_96,pc_97,pc_98,pc_99,pc_100,pc_101,pc_102,pc_103,pc_104,pc_105,pc_106,pc_107,pc_108,pc_109,pc_110,pc_111,pc_112,pc_113,pc_114,pc_115,pc_116,pc_117
0,abaiara,0.909037,1.411531,-2.009080,1.273723,-4.484937,0.928649,2.576158,-0.231018,-0.637624,-2.344075,1.828871,2.524196,-0.455517,-0.179312,-0.563414,2.679475,0.417502,-0.294738,-0.933825,0.305719,-0.684745,0.864986,-1.443222,-0.188149,-1.024488,-1.527550,-1.217833,-0.716385,-0.763641,0.155937,-0.186825,-1.305830,1.615400,-0.219660,0.499044,-1.764376,-0.413343,-0.945728,-0.311493,-0.158862,1.696927,0.417107,0.390798,0.275250,-0.216778,-0.651223,-0.194677,1.123314,1.416144,-1.291248,0.438839,-0.534997,-0.050953,1.834906,-0.609948,-0.027921,-0.180196,0.420107,-0.146671,0.478700,-0.303394,0.005258,-0.113624,0.079117,-0.041201,0.229572,0.300062,0.031663,-0.438545,-0.265928,0.208702,-0.148927,0.314288,-0.112894,-0.367687,0.275302,-0.661852,-0.299656,0.013105,0.491303,0.089097,0.157765,0.034975,-0.065527,-0.575389,-0.103143,0.021064,-0.003109,-0.004513,-0.061911,0.100914,0.017509,-0.014824,-0.028519,0.021739,0.001316,0.000682,-0.000333,-0.000664,-0.000400,-0.000509,0.000345,0.000127,1.448631e-05,-0.000138,3.081238e-04,-0.000247,-1.506733e-04,-6.036117e-06,-0.000025,0.000136,2.648380e-13,-4.099649e-14,4.678895e-16,-1.651873e-16,-2.123939e-16,-3.785838e-16
1,acarape,-0.199619,-3.821894,4.499563,-0.162677,0.529191,0.706435,-0.765329,-3.102003,1.522149,2.087833,-3.617069,1.292302,2.601272,-2.131336,-0.161877,0.517940,3.820060,2.170087,1.206195,-1.560534,0.751207,-0.017205,-0.032034,-0.353542,1.121924,0.578828,-1.553725,-0.049419,-1.112927,2.821912,0.766232,0.656711,-0.542350,0.225125,0.439106,-1.100519,-2.013145,0.391836,0.325904,0.302403,-0.788717,-1.071058,-0.739985,2.275477,-0.145049,-0.240876,-1.550864,0.396717,0.300441,0.761697,0.479144,-1.010247,-0.397769,0.578299,-0.844085,-0.098276,-0.217387,-0.896503,-0.376577,0.580311,-0.930283,-0.046268,0.051611,-0.397456,-0.092882,-0.183327,0.849214,0.373739,0.525688,0.448857,0.883268,-0.390974,-0.169754,0.014421,-0.149940,-0.099265,0.043338,-0.546803,-0.179020,-0.242604,-0.336244,-0.123581,-0.075698,-0.473959,-0.044005,-0.202235,0.127592,-0.136044,0.088384,0.146643,-0.009607,0.070559,-0.054995,-0.011812,0.016872,-0.000863,-0.000215,0.000369,-0.000007,0.000277,-0.000425,0.000013,-0.000336,-4.101556e-04,-0.000202,-1.087290e-04,-0.000058,1.345430e-04,5.753147e-05,-0.000055,0.000121,-2.312815e-13,7.337910e-15,2.682893e-16,1.160496e-17,3.006351e-16,-3.395572e-16
2,acarau,-5.889520,-3.462552,-0.796370,0.579979,2.197895,-1.147328,2.309388,2.006473,-0.501712,0.325099,-0.615748,0.104987,-0.812516,2.556157,1.699051,0.539604,1.061833,0.581043,-0.810778,0.716949,-0.967842,-0.416273,-0.316289,-1.320935,-0.118191,0.401969,-1.131552,-0.236336,-0.011506,-0.569544,0.964772,0.683596,0.259770,0.168286,0.958485,-0.723991,-0.879174,1.644171,0.511262,-0.652095,-0.614723,-1.299357,0.002977,-0.382724,-0.579747,-1.285552,0.385443,-0.424022,0.208459,0.687161,0.136973,0.580427,-0.050148,-0.483109,-0.465412,-0.146955,-0.478983,-0.820840,0.739533,0.724582,0.068230,-0.105278,0.097833,0.579700,-0.830815,-0.338152,-0.021876,0.380102,-0.168917,0.435595,-0.202996,-0.151245,-0.579793,0.286272,-0.322500,-0.189177,0.075943,0.174631,0.181630,-0.219291,-0.252763,0.388024,0.005805,0.205751,0.153734,-0.331493,0.150896,-0.082271,-0.007261,0.193542,-0.074106,-0.124129,0.004991,-0.004787,-0.016041,0.002144,-0.000151,0.0010

In [238]:
cadunico_pca.to_csv('/content/drive/MyDrive/Hunger prediction/data/cadunico_pca.csv', index = False)

### 2.1.1. Análise do desempenho da PCA a partir de matrizes de correlção

In [239]:
def histogram_plot(mtx, width=10000, height=600):
  fig = px.imshow(mtx[:][-4:],
                #x=cadunico_data.columns,
                #y=cadunico_data.columns,
                text_auto=True,
                color_continuous_scale='Inferno',
                width=width, height=height
               )

  fig.update_layout(
      margin=dict(l=0, r=0, t=0, b=0),
      autosize=True,
      #xaxis_nticks=len(cadunico_data.columns),
      #yaxis_nticks=len(cadunico_data.columns)
      )

  fig.show()

In [240]:
data = cadunico_pca.join(sisvan.set_index('Município'), on = 'Município', how = 'left')
data = data[data.columns[1:]].copy()
corr_matrix = data.corr()

In [241]:
histogram_plot(corr_matrix)

In [242]:
spear_corr = data.corr(method="spearman")

In [243]:
histogram_plot(spear_corr)

In [244]:
data2 = cadunico.join(sisvan.set_index('Município'), on = 'Município', how = 'left')
data2 = data2[data2.columns[1:]].copy()
corr_matrix = data2.corr()

In [245]:
histogram_plot(data2.corr(), 14000, 800)

In [246]:
histogram_plot(data2.corr(method="spearman"), 14000, 800)

## 2.2. Tratamento da assimetria

Esse tratamento só é pssível após a aolicação do PCA, não sei exatamente porque, mas se houver a aplicação do método de Yeo-Johnson sem a PCA as amostras transformadas se tornam todas nulas.

In [247]:
cadunico_pca.skew()

<ipython-input-247-f418006f283c>:1: FutureWarning:

The default value of numeric_only in DataFrame.skew is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



pc_1      0.500942
pc_2     -0.070616
pc_3      0.403457
pc_4      0.635837
pc_5     -0.060934
pc_6      0.261280
pc_7      0.525873
pc_8      0.209859
pc_9      0.943990
pc_10     0.899692
pc_11     0.838633
pc_12     0.500912
pc_13     1.308478
pc_14     0.264821
pc_15     1.213984
pc_16     0.446860
pc_17     0.484954
pc_18     0.705730
pc_19     0.755462
pc_20     0.299220
pc_21     0.833817
pc_22     0.320638
pc_23     0.372062
pc_24     0.305667
pc_25     0.634396
pc_26     0.646098
pc_27     0.394259
pc_28     0.245658
pc_29     0.236281
pc_30     0.233717
pc_31     0.436588
pc_32     0.193566
pc_33     0.105862
pc_34     0.489420
pc_35     0.251240
pc_36    -0.054982
pc_37     0.173320
pc_38     0.163804
pc_39     0.073837
pc_40     0.159665
pc_41     0.106592
pc_42     0.270876
pc_43     0.618546
pc_44     0.643220
pc_45     0.221709
pc_46     0.112821
pc_47     0.013797
pc_48     0.124863
pc_49     0.272502
pc_50     0.089964
pc_51     0.064069
pc_52     0.176627
pc_53     0.

In [ ]:
cadunico_pca.hist(
    figsize=(40, 300),
    grid = False,
    rwidth = 0.9,
    bins = 80,
    layout = (40,3)
)

plt.show()

In [ ]:
test_data = cadunico_pca[cadunico_pca.columns[1:]].copy()
lambs = []

for col in cadunico_pca.columns[1:]:
  if abs(test_data[col].skew()) > 0.75:
    print(col)
    # lambda_optimal = find_best_lambda(test_data[col], lambda_range)
    test_data[col], l = stats.yeojohnson(test_data[col])[0:2]
    lambs.append(l)

In [ ]:
lambs

In [ ]:
test_data.skew()

In [252]:
test_data.hist(
    figsize=(40, 300),
    grid = False,
    rwidth = 0.9,
    bins = 80,
    layout = (40,3)
)

plt.show()

In [253]:
test_data['Município'] = cadunico_pca['Município']

In [254]:
data3 = test_data.join(sisvan.set_index('Município'), on = 'Município', how = 'left')
data3 = data3[data3.columns[1:]].copy()
corr_matrix = data3.corr()

<ipython-input-254-f33d4bc9956e>:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [255]:
histogram_plot(data3.corr(), 14000, 800)

<ipython-input-255-29f899b4cd0c>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [256]:
histogram_plot(data3.corr(method="spearman"), 14000, 800)

<ipython-input-256-10d766594898>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

